<a href="https://colab.research.google.com/github/areias/bert_covid_sentiment/blob/main/bert_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Training

from https://github.com/digitalepidemiologylab/covid-twitter-bert/blob/c87912b409659f40018e839c4124be5ae2486713/run_finetune.py


In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!ls

drive  sample_data


In [3]:
#!git clone -b master https://github.com/digitalepidemiologylab/covid-twitter-bert.git

In [4]:
!pip install -r drive/MyDrive/covid-twitter-bert/requirements.txt

     |████████████████████████████████| 516.2 MB 4.2 kB/s 
     |████████████████████████████████| 1.1 MB 58.1 MB/s 
     |████████████████████████████████| 170 kB 88.2 MB/s 
     |████████████████████████████████| 235 kB 59.5 MB/s 
     |████████████████████████████████| 1.2 MB 63.4 MB/s 
     |████████████████████████████████| 454 kB 65.3 MB/s 
     |████████████████████████████████| 3.0 MB 54.1 MB/s 
     |████████████████████████████████| 2.9 MB 55.6 MB/s 
     |████████████████████████████████| 57 kB 6.4 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=c559a95ce453937c85a3d0235c9cd239477a6c30a46fbac4b33361654ebac2d8
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estima

In [5]:
import tensorflow
print(tensorflow.__version__)

2.2.0


In [6]:
import sys
sys.path.append('drive/MyDrive/covid-twitter-bert')
sys.path.append('drive/MyDrive/covid-twitter-bert/tensorflow_models')


In [7]:
from official.nlp.bert import bert_models
from official.utils.misc import distribution_utils
from official.nlp.bert import configs as bert_configs
from official.modeling import performance
from official.nlp.bert import input_pipeline
from official.utils.misc import keras_utils

In [8]:
import os
import datetime
import time
import argparse
import math
import logging
from logging.handlers import RotatingFileHandler


In [74]:
import tqdm
import json
import tensorflow as tf
from utils.misc import ArgParseDefault, save_to_json, add_bool_arg
from utils.finetune_helpers import Metrics
import utils.optimizer
from config import PRETRAINED_MODELS

In [79]:
import importlib
importlib.reload(__import__('config')); from config import PRETRAINED_MODELS
PRETRAINED_MODELS

{'bert_large_uncased': {'bucket_location': 'pretrained_models/bert/keras_bert/uncased_L-24_H-1024_A-16',
  'config': 'bert_config_large_uncased.json',
  'do_whole_word_masking': False,
  'hub_url': 'tensorflow/bert_en_uncased_L-24_H-1024_A-16/2',
  'is_tfhub_model': True,
  'lower_case': True,
  'vocab_file': 'bert-large-uncased-vocab.txt'},
 'bert_large_uncased_wwm': {'bucket_location': 'pretrained_models/bert/keras_bert/wwm_uncased_L-24_H-1024_A-16',
  'config': 'bert_config_large_uncased_wwm.json',
  'do_whole_word_masking': True,
  'hub_url': 'tensorflow/bert_en_wwm_uncased_L-24_H-1024_A-16/2',
  'is_tfhub_model': True,
  'lower_case': True,
  'vocab_file': 'bert-large-uncased-whole-word-masking-vocab.txt'},
 'bert_multi_cased': {'bucket_location': 'pretrained_models/bert/keras_bert/multi_cased_L-12_H-768_A-12',
  'config': 'bert_config_multi_cased.json',
  'do_whole_word_masking': False,
  'hub_url': 'tensorflow/bert_multi_cased_L-12_H-768_A-12/2',
  'is_tfhub_model': True,
  'low

In [11]:
import datetime
def get_run_name():
    # Use timestamp to generate a unique run name
    ts = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S_%f')
    run_name = f'run_{ts}'
    return run_name


In [ ]:
"""python run_finetune.py \
  --run_prefix $RUN_PREFIX \
  --bucket_name $BUCKET_NAME \
  --tpu_ip $TPU_IP \
  --model_class $MODEL_CLASS \
  --finetune_data ${FINETUNE_DATA}/${FINETUNE_DATASET} \
  --train_batch_size $TRAIN_BATCH_SIZE \
  --eval_batch_size $EVAL_BATCH_SIZE \
  --num_epochs $NUM_EPOCHS \
  --learning_rate $LR"""



'python run_finetune.py   --run_prefix $RUN_PREFIX   --bucket_name $BUCKET_NAME   --tpu_ip $TPU_IP   --model_class $MODEL_CLASS   --finetune_data ${FINETUNE_DATA}/${FINETUNE_DATASET}   --train_batch_size $TRAIN_BATCH_SIZE   --eval_batch_size $EVAL_BATCH_SIZE   --num_epochs $NUM_EPOCHS   --learning_rate $LR'

In [16]:
# args

RUN_PREFIX='testrun'                                  # Name your run
#BUCKET_NAME=                                        # Fill in your buckets name here (without the gs:// prefix)
#TPU_IP=XX.XX.XXX.X                                  # Fill in your TPUs IP here
FINETUNE_DATASET='crowdbreaks'                      # Your dataset name
FINETUNE_DATA= 'run_2021-11-24_14-56-07_010193_test_run'                        # Fill in dataset run name (e.g. run_2020-05-19_14-14-53_517063_test_run)
MODEL_CLASS='covid-twitter-bert'
TRAIN_BATCH_SIZE=8 #32b
EVAL_BATCH_SIZE=8
LR=2e-5
NUM_EPOCHS=1

In [17]:
from collections import namedtuple
arguments = namedtuple('arguments', ['run_prefix','model_class','finetune_data', 
                                     'train_batch_size', 'eval_batch_size','learning_rate',
                                     'limit_train_steps','limit_eval_steps','num_epochs',
                                     'warmup_proportion', 'init_checkpoint','validation_freq',
                                     'end_lr','optimizer_type', 'save_model',
                                     'early_stopping_epochs','time_history_log_steps'])

args = arguments(RUN_PREFIX,MODEL_CLASS,FINETUNE_DATA,
                 TRAIN_BATCH_SIZE,EVAL_BATCH_SIZE,LR,
                 None,None,1,
                 0.1, None,1,
                 0,'adamw',True,
                1,10)            
args 

arguments(run_prefix='testrun', model_class='covid-twitter-bert', finetune_data='run_2021-11-24_14-56-07_010193_test_run', train_batch_size=8, eval_batch_size=8, learning_rate=2e-05, limit_train_steps=None, limit_eval_steps=None, num_epochs=1, warmup_proportion=0.1, init_checkpoint=None, validation_freq=1, end_lr=0, optimizer_type='adamw', save_model=True, early_stopping_epochs=1, time_history_log_steps=10)

In [18]:
def set_mixed_precision_policy(args):
    """Sets mix precision policy."""
    if args.dtype == 'fp16':
        policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16', loss_scale=loss_scale)
        tf.keras.mixed_precision.experimental.set_policy(policy)
    elif args.dtype == 'bf16':
        policy = tf.keras.mixed_precision.experimental.Policy('mixed_bfloat16')
        tf.keras.mixed_precision.experimental.set_policy(policy)
    elif args.dtype == 'fp32':
        tf.keras.mixed_precision.experimental.set_policy('float32')
    else:
        raise ValueError(f'Unknown dtype {args.dtype}')

In [39]:
#def run(args):
"""Train using the Keras/TF 2.0. Adapted from the tensorflow/models Github"""
# CONFIG
run_name = get_run_name()
#logger.info(f'*** Starting run {run_name} ***')
data_dir = 'drive/MyDrive/covid-twitter-bert/data/finetune/'+args.finetune_data+'/crowdbreaks/'
output_dir = 'drive/MyDrive/covid-twitter-bert/data/finetune/'+args.finetune_data+'/crowdbreaks/'+run_name

output_dir

'drive/MyDrive/covid-twitter-bert/data/finetune/run_2021-11-24_14-56-07_010193_test_run/crowdbreaks/run_2021-11-24_15-23-04_035242'

In [40]:
def get_model_config(config_path):
    config = bert_configs.BertConfig.from_json_file(config_path)
    return config

In [41]:
PRETRAINED_MODELS[args.model_class]['config']

'bert_config_covid_twitter_bert.json'

In [58]:
def get_model_config_path(args):
    try:
        config_path = PRETRAINED_MODELS[args.model_class]['config']
    except KeyError:
        raise ValueError(f'Could not find a pretrained model matching the model class {args.model_class}')
    return os.path.join('drive/MyDrive/covid-twitter-bert/configs/', config_path)

In [80]:
# Get configs
pretrained_model_config_path = get_model_config_path(args)
model_config = get_model_config(pretrained_model_config_path)

In [81]:
pretrained_model_config_path

'drive/MyDrive/covid-twitter-bert/configs/bert_config_covid_twitter_bert.json'

In [82]:
model_config

In [83]:
def get_input_meta_data(data_dir):
    with tf.io.gfile.GFile(data_dir+'/meta.json', 'rb') as reader:
        input_meta_data = json.loads(reader.read().decode('utf-8'))
    return input_meta_data


In [84]:
def get_label_mapping(data_dir):
    with tf.io.gfile.GFile(data_dir+'/label_mapping.json', 'rb') as reader:
        label_mapping = json.loads(reader.read().decode('utf-8'))
    label_mapping = dict(zip(range(len(label_mapping)), label_mapping))
    return label_mapping

In [85]:
# Meta data/label mapping
input_meta_data = get_input_meta_data(data_dir)
label_mapping = get_label_mapping(data_dir)
#logger.info(f'Loaded training data meta.json file: {input_meta_data}')

In [86]:


input_meta_data

{'eval_data_size': 680,
 'max_seq_length': 96,
 'num_labels': 3,
 'processor_type': 'text-classification',
 'task_type': 'bert_classification',
 'train_data_size': 2041}

In [87]:
label_mapping

{0: -1, 1: 0, 2: 1}

In [88]:
# Calculate steps, warmup steps and eval steps
train_data_size = input_meta_data['train_data_size']
num_labels = input_meta_data['num_labels']
max_seq_length = input_meta_data['max_seq_length']
if args.limit_train_steps is None:
    steps_per_epoch = int(train_data_size / args.train_batch_size)
else:
    steps_per_epoch = args.limit_train_steps
warmup_steps = int(args.num_epochs * train_data_size * args.warmup_proportion/ args.train_batch_size)
if args.limit_eval_steps is None:
    eval_steps = int(math.ceil(input_meta_data['eval_data_size'] / args.eval_batch_size))
else:
    eval_steps = args.limit_eval_steps

In [89]:
def get_model(args, model_config, steps_per_epoch, warmup_steps, num_labels, max_seq_length, is_hub_module=False):
    # Get classifier and core model (used to initialize from checkpoint)
    if args.init_checkpoint is None and PRETRAINED_MODELS[args.model_class]['is_tfhub_model']:
        # load pretrained model from TF-hub
        hub_module_url = f"https://tfhub.dev/{PRETRAINED_MODELS[args.model_class]['hub_url']}"
        hub_module_trainable = True
    else:
        hub_module_url = None
        hub_module_trainable = False
    classifier_model, core_model = bert_models.classifier_model(
            model_config,
            num_labels,
            max_seq_length,
            hub_module_url=hub_module_url,
            hub_module_trainable=hub_module_trainable)
    # Optimizer
    optimizer = utils.optimizer.create_optimizer(
            args.learning_rate,
            steps_per_epoch * args.num_epochs,
            warmup_steps,
            args.end_lr,
            args.optimizer_type)
    classifier_model.optimizer = configure_optimizer(
            optimizer,
            use_float16=False,
            use_graph_rewrite=False)
    return classifier_model, core_model

In [90]:
def configure_optimizer(optimizer, use_float16=False, use_graph_rewrite=False, loss_scale='dynamic'):
    """Configures optimizer object with performance options."""
    if use_float16:
        # Wraps optimizer with a LossScaleOptimizer. This is done automatically in compile() with the
        # "mixed_float16" policy, but since we do not call compile(), we must wrap the optimizer manually.
        optimizer = (tf.keras.mixed_precision.experimental.LossScaleOptimizer(optimizer, loss_scale=loss_scale))
    if use_graph_rewrite:
        # Note: the model dtype must be 'float32', which will ensure
        # tf.ckeras.mixed_precision and tf.train.experimental.enable_mixed_precision_graph_rewrite do not double up.
        optimizer = tf.train.experimental.enable_mixed_precision_graph_rewrite(optimizer)
    return optimizer

In [91]:
def get_loss_fn(num_classes):
    """Gets the classification loss function."""
    def classification_loss_fn(labels, logits):
        """Classification loss."""
        labels = tf.squeeze(labels)
        log_probs = tf.nn.log_softmax(logits, axis=-1)
        one_hot_labels = tf.one_hot(tf.cast(labels, dtype=tf.int32), depth=num_classes, dtype=tf.float32)
        per_example_loss = -tf.reduce_sum(tf.cast(one_hot_labels, dtype=tf.float32) * log_probs, axis=-1)
        return tf.reduce_mean(per_example_loss)
    return classification_loss_fn

In [92]:
# Get model
classifier_model, core_model = get_model(args, model_config, steps_per_epoch, warmup_steps, num_labels, max_seq_length)
optimizer = classifier_model.optimizer
loss_fn = get_loss_fn(num_labels)
#try:
#    if ',' in args.validation_freq:
#        validation_freq = args.validation_freq.split(',')
#        validation_freq = [int(v) for v in validation_freq]
#    else:
validation_freq = int(args.validation_freq)
#except:
#    raise ValueError(f'Invalid argument for validation_freq!')
#logger.info(f'Using a validation frequency of {validation_freq}')


In [93]:
classifier_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 96)]         0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 96)]         0                                            
__________________________________________________________________________________________________
input_type_ids (InputLayer)     [(None, 96)]         0                                            
__________________________________________________________________________________________________
keras_layer_2 (KerasLayer)      [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]           

In [94]:
classifier_model.get_config()

{'input_layers': {'input_mask': ['input_mask', 0, 0],
  'input_type_ids': ['input_type_ids', 0, 0],
  'input_word_ids': ['input_word_ids', 0, 0]},
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 96),
    'dtype': 'int32',
    'name': 'input_word_ids',
    'ragged': False,
    'sparse': False},
   'inbound_nodes': [],
   'name': 'input_word_ids'},
  {'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 96),
    'dtype': 'int32',
    'name': 'input_mask',
    'ragged': False,
    'sparse': False},
   'inbound_nodes': [],
   'name': 'input_mask'},
  {'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 96),
    'dtype': 'int32',
    'name': 'input_type_ids',
    'ragged': False,
    'sparse': False},
   'inbound_nodes': [],
   'name': 'input_type_ids'},
  {'class_name': 'KerasLayer',
   'config': {'dtype': 'float32',
    'handle': 'https://tfhub.dev/digitalepidemiologylab/covid-twitter-bert/2',
    'name': 'keras_layer_2

In [96]:
def get_metrics():
    return [tf.keras.metrics.SparseCategoricalAccuracy('accuracy', dtype=tf.float32)]


In [97]:
# Run keras compile
#logger.info(f'Compiling keras model...')
classifier_model.compile(
    optimizer=optimizer,
    loss=loss_fn,
    metrics=get_metrics())
#logger.info(f'... done')

In [98]:
# Create all custom callbacks
summary_dir = os.path.join(output_dir, 'summaries')
summary_callback = tf.keras.callbacks.TensorBoard(summary_dir, profile_batch=0)
time_history_callback = keras_utils.TimeHistory(
    batch_size=args.train_batch_size,
    log_steps=args.time_history_log_steps,
    logdir=summary_dir)
custom_callbacks = [summary_callback, time_history_callback]
if args.save_model:
    #logger.info('Using save_model option...')
    checkpoint_path = os.path.join(output_dir, 'checkpoint')
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, verbose=1)
    custom_callbacks.append(checkpoint_callback)
if args.early_stopping_epochs > 0:
    #logger.info(f'Using early stopping of after {args.early_stopping_epochs} epochs of val_loss not decreasing')
    early_stopping_callback = tf.keras.callbacks.EarlyStopping(patience=args.early_stopping_epochs, monitor='val_loss')
    custom_callbacks.append(early_stopping_callback)

In [99]:
def get_dataset_fn(input_file_pattern, max_seq_length, global_batch_size, is_training=True):
  """Gets a closure to create a dataset."""
  def _dataset_fn(ctx=None):
    """Returns tf.data.Dataset for distributed BERT pretraining."""
    batch_size = ctx.get_per_replica_batch_size(
        global_batch_size) if ctx else global_batch_size
    dataset = input_pipeline.create_classifier_dataset(
        input_file_pattern,
        max_seq_length,
        batch_size,
        is_training=is_training,
        input_pipeline_context=ctx)
    return dataset

  return _dataset_fn

In [100]:
# Generate dataset_fn
train_input_fn = get_dataset_fn(
    data_dir+'/tfrecords/train.tfrecords',
    max_seq_length,
    args.train_batch_size,
    is_training=True)
eval_input_fn = get_dataset_fn(
    data_dir+'/tfrecords/dev.tfrecords',
    max_seq_length,
    args.eval_batch_size,
    is_training=False)


In [101]:
data_dir

'drive/MyDrive/covid-twitter-bert/data/finetune/run_2021-11-24_14-56-07_010193_test_run/crowdbreaks/'

In [102]:
# Add mertrics callback to calculate performance metrics at the end of epoch
performance_metrics_callback = Metrics(
        eval_input_fn,
        label_mapping,
        os.path.join(summary_dir, 'metrics'),
        eval_steps,
        args.eval_batch_size,
        validation_freq)
custom_callbacks.append(performance_metrics_callback)

In [103]:
# Run keras fit
time_start = time.time()
#logger.info('Run training...')
history = classifier_model.fit(
    x=train_input_fn(),
    validation_data=eval_input_fn(),
    steps_per_epoch=steps_per_epoch,
    epochs=args.num_epochs,
    validation_steps=eval_steps,
    validation_freq=validation_freq,
    callbacks=custom_callbacks,
    verbose=1)
time_end = time.time()
training_time_min = (time_end-time_start)/60
#logger.info(f'Finished training after {training_time_min:.1f} min')


255/255 [==============================] - ETA: 0s - loss: 1.0367 - accuracy: 0.4804
Epoch 00001: saving model to drive/MyDrive/covid-twitter-bert/data/finetune/run_2021-11-24_14-56-07_010193_test_run/crowdbreaks/run_2021-11-24_15-23-04_035242/checkpoint
255/255 [==============================] - 143s 561ms/step - loss: 1.0367 - accuracy: 0.4804 - val_loss: 0.8922 - val_accuracy: 0.6412


In [104]:
# Write training log
all_scores = performance_metrics_callback.scores
all_predictions = performance_metrics_callback.predictions
if len(all_scores) > 0:
    final_scores = all_scores[-1]
    #logger.info(f'Final eval scores: {final_scores}')
else:
    final_scores = {}
full_history = history.history
if len(full_history) > 0:
    final_val_loss = full_history['val_loss'][-1]
    final_loss = full_history['loss'][-1]
    #logger.info(f'Final training loss: {final_loss:.2f}, Final validation loss: {final_val_loss:.2f}')
else:
    final_val_loss = None
    final_loss = None
data = {
        'created_at': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'run_name': run_name,
        'final_loss': final_loss,
        'final_val_loss': final_val_loss,
        'max_seq_length': max_seq_length,
        'num_train_steps': steps_per_epoch * args.num_epochs,
        'eval_steps': eval_steps,
        'steps_per_epoch': steps_per_epoch,
        'training_time_min': training_time_min,
        'data_dir': data_dir,
        'output_dir': output_dir,
        'all_scores': all_scores,
        'all_predictions': all_predictions,
        'num_labels': num_labels,
        'label_mapping': label_mapping,
        **full_history,
        **final_scores,
        'args':args}

In [105]:
os.path.join(output_dir, run_name, 'run_logs.json')

'drive/MyDrive/covid-twitter-bert/data/finetune/run_2021-11-24_14-56-07_010193_test_run/crowdbreaks/run_2021-11-24_15-23-04_035242/run_2021-11-24_15-23-04_035242/run_logs.json'

In [109]:
# Write run_log
f_path_training_log = os.path.join(output_dir, 'run_logs.json')
#logger.info(f'Writing training log to {f_path_training_log}...')
save_to_json(data, f_path_training_log)
# Write bert config
model_config.id2label = label_mapping
model_config.label2id = {v:k for k, v in label_mapping.items()}
model_config.max_seq_length = max_seq_length
model_config.num_labels = num_labels
f_path_bert_config = os.path.join(output_dir, 'bert_config.json')
#logger.info(f'Writing BERT config to {f_path_bert_config}...')
save_to_json(model_config.to_dict(), f_path_bert_config)